## 手动搭建一个RNN网络

In [1]:
import sys
sys.path.append('../')

In [2]:
import gluonbook as gb
import math
from mxnet import autograd, nd
from mxnet.gluon import loss as gloss
import time
import mxnet as mx

C:\Users\lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
(corpus_indices, char_to_idx, idx_to_char,
vocab_size) = gb.load_data_jay_lyrics()

## One-hot向量

In [4]:
nd.one_hot(nd.array([0, 2],ctx=mx.gpu()), vocab_size,)


[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
<NDArray 2x1027 @gpu(0)>

由于输入样本是 [批量大小，时间步长] ， 我们需要将其转化为One-Hot编码，编码后格式为 [时间步长，（批量大小，词典词总数）]，即每个批量都是一个One-hot编码，训练时，有步长次输入

In [5]:
def ont_hot_encoder(X,size):
    return [nd.one_hot(word,size) for word in X.T]

In [6]:
X = nd.arange(10,ctx=mx.gpu()).reshape((2, 5))
inputs = ont_hot_encoder(X, vocab_size)
len(inputs), inputs[0].shape

(5, (2, 1027))

In [7]:
num_inputs,num_hiddens,num_outputs = vocab_size,256,vocab_size

In [8]:
ctx = gb.try_gpu()

In [9]:
def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01,shape=shape,ctx=ctx)
    
    #隐藏层参数
    W_xh = nd.random.normal(scale=0.01,shape=(num_inputs,num_hiddens),ctx=ctx)
    W_hh = nd.random.normal(scale=0.01,shape=(num_hiddens,num_hiddens),ctx=ctx)
    b_h = nd.zeros(shape=(num_hiddens,),ctx=ctx)
    W_ho = nd.random.normal(scale=0.01,shape=(num_hiddens,num_outputs),ctx=ctx)
    b_o = nd.zeros(shape=(num_outputs,),ctx=ctx)
    
    #申请梯度求导内存
    params=[W_xh,W_hh,b_h,W_ho,b_o]
    for param in params:
        param.attach_grad()
    return params

## 定义rnn模型

+ ⾸先定义 init_rnn_state 函数来返回初始化的隐藏状态。它返回由⼀个形状为（批量⼤小，隐藏单元个数）的值为 0 的 NDArray 组成的循环神经⽹络元组。使⽤元组是为了更⽅便处理隐藏状态含有多个 NDArray 的情况

In [10]:
def init_rnn_state(batch_size,num_hiddens,ctx):
    return (nd.zeros(shape=(batch_size,num_hiddens),ctx=ctx),)

## 下面定义rnn的计算

In [11]:
def rnn(input,state,params):
    # inputs 和 outputs 皆为 num_steps 个形状为（batch_size， vocab_size）的矩阵。
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, =state
    outputs = []
    for X in input:
        H = nd.relu(nd.dot(X,W_xh)+nd.dot(H,W_hh)+b_h)
        Y = nd.dot(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,)

In [12]:
state=init_rnn_state(X.shape[0],num_hiddens,ctx)
input = ont_hot_encoder(X,vocab_size)
params = get_params()
outputs,new_states = rnn(input,state,params)


In [13]:
len(outputs),outputs[0].shape,new_states[0].shape

(5, (2, 1027), (2, 256))

## 定义预测函数

+ 函数基于prefix个字符，来预测接下来的num_chars个字符

In [14]:
def predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,
                num_hiddens,vocab_size,ctx,idx_to_char,char_to_idx):
    #首先预处理输入数据,一个输入换成索引
    output = [char_to_idx[prefix[0]]]
    #初始化隐藏城状态
    state = init_rnn_state(1,num_hiddens,ctx)  #批量数为1
    
    for t in range(num_chars+len(prefix)-1):
        #将上一次的输出作为这一次的输入
        X = ont_hot_encoder(nd.array([output[-1]],ctx=ctx),vocab_size)
        #前向运算，获得输出
        (Y,state) = rnn(X,state,params)
        #将输出追加到最后
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            #这里 Y 要取下标0是由于 Y为一个列表，需要取出元素才能运算
            output.append(int(Y[0].argmax(axis=1).asscalar()))
    
    return ''.join([idx_to_char[i] for i in output])

In [15]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
ctx, idx_to_char, char_to_idx)

'分开四彿去始草泪剧祭去宠'

## 裁剪梯度

 * 循环神经网络汇总容易出现梯度衰减或梯度爆炸。所以我们把所有模型参数梯度的元素拼接成一个向量 $\boldsymbol{g}$,并设置裁剪的阈值为 $\theta$。裁剪后的梯度为
 
 $$
 \min \left({\frac{\theta}{||\boldsymbol{g}||},1}\right)\boldsymbol{g}
 $$
 
的$L_2$范数不超过$\theta$

In [16]:
def grad_clipping(params,theta,ctx):
    norm = nd.array([0.0],ctx=ctx)
    for param in params:
        norm+=(param.grad**2).sum()
    norm = norm.sqrt().asscalar()
    
    if norm>theta:
        for param in params:
            param.grad[:] *= theta/norm 

## 困惑度

* 困惑度是对交叉熵损失函数做指数运算后得到的值。特别的：
    - 最佳情况下，模型总是把标签类别的概率预测为 1。此时困惑度为 1
    - 最坏情况下，模型总是把标签类别的概率预测为 0。此时困惑度为正⽆穷。
    - 基线情况下，模型总是预测所有类别的概率都相同。此时困惑度为类别个数。
    
显然，任何⼀个有效模型的困惑度必须小于类别个数。在本例中，困惑度必须小于词典⼤小 vocab_size。

## 定义模型训练函数

1. 使用困惑度(perplexity)来评价模型
2. 在迭代模型参数前裁剪梯度
3. 对时序数据采用不同采样方法将导致隐层状态初始化的不同

In [17]:
import time
def train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,
                         vocab_size,ctx,corpus_indices,idx_to_char,
                         char_to_idx,is_random_iter,num_epochs,num_steps,lr,
                         clipping_theta,batch_size,pred_period,pred_len,prefixes):
    if is_random_iter:
        data_iter_fn = gb.data_iter_random
    else:
        data_iter_fn = gb.data_iter_consecutive
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter:             #如果使用相邻采样，则在epoch开始时初始化隐藏层状态
            state = init_rnn_state(batch_size,num_hiddens,ctx)
        loss_sum,start = 0.0,time.time()
        data_iter = data_iter_fn(corpus_indices,batch_size,num_steps,ctx)
        for t,(X,y) in enumerate(data_iter):
            if is_random_iter:  #如果是随机采样，在每个小批量更新前初始化隐藏层状态
                state = init_rnn_state(batch_size,num_hiddens,ctx)
            else: # 否则需要使⽤ detach 函数从计算图分离隐藏状态。
                for s in state:
                    s.detach()
            #下面进行训练
            with autograd.record():
                #首先 one-hot编码
                inputs = ont_hot_encoder(X,vocab_size)
                # outputs 有 num_steps 个形状为（batch_size， vocab_size）的矩阵。
                (outputs,state) = rnn(inputs,state,params)
                outputs = nd.concat(*outputs,dim=0)
                # Y 的形状是（batch_size， num_steps），转置后再变成⻓度为
                # batch * num_steps 的向量，这样跟输出的⾏⼀⼀对应。
                y =y.T.reshape((-1,))
                l = loss(outputs,y).mean()
            l.backward()
            grad_clipping(params,clipping_theta,ctx)
            gb.sgd(params,lr,1)
            loss_sum += l.asscalar()
        if (epoch+1) % pred_period==0:
            print('epoch %d,perplexity %f,time %.2fsec'%
                 (epoch+1,math.exp(loss_sum/(t+1)),time.time()-start))
            for prefix in prefixes:
                print('-',predict_rnn(
                prefix,pred_len,rnn,params,init_rnn_state,num_hiddens,vocab_size,ctx,idx_to_char,char_to_idx))

In [18]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 1000, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 100, 50, ['分开', '不分开']

## 随机采样进行训练

In [19]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
vocab_size, ctx, corpus_indices, idx_to_char,
char_to_idx, True, num_epochs, num_steps, lr,
clipping_theta, batch_size, pred_period, pred_len,
prefixes)

epoch 100,perplexity 17.165624,time 0.50sec
- 分开 干什么我有朋友一起 融化的让旧人多 牧草有没有 我马儿有些瘦你久跑像人 你我的让我面疼的可爱女人 
- 不分开 漂我的恨界里多苍牧哭有悲害 我给儿这生奏就久的手不放多的爱头有 用括的老斑人 印地正不的牛肉 我说
epoch 200,perplexity 3.018563,time 0.49sec
- 分开 一直令它心仪的母斑鸠缸牛爷爷玫 我失轻的叹尾 不通里这样的屋内 还底什么我想要 却发现迷了路怎么找
- 不分开期 风后将不了口慢  我教的有 它阵莫名念动翰福音现为弥补 心有一双蓝色眼睛斑脸天方决斗 伤养上黑一
epoch 300,perplexity 1.780099,time 0.51sec
- 分开 一只令它三仪的母斑鸠 印像一阵风 吹完它就走 这样的让我面红的可爱女人 温柔的让我心疼的可爱女人 
- 不分开期 我叫你爸 你打我妈 这样的吗干都的晴 随 让午来飞子光 瞎 说们都的风迹 不悔成被忆 不街茶美 
epoch 400,perplexity 1.581123,time 0.57sec
- 分开 一只令它三仪七百多年 灰袋橱有一点秀逗 猎物让夕它飞被走难进y堂的角度 能知道你前世是狼人还道 你
- 不分开扫 然后将过去 慢慢温习 让我爱上你 那场悲剧 是你完美演出的一场戏 宁愿心碎哭泣 再狠狠忘记 你爱
epoch 500,perplexity 1.493163,time 0.50sec
- 分开 距三已 一手走 我打就这样牵着你的手不放开 爱可不能够永远单纯没有悲哀 你 靠着我的肩膀 你 在我
- 不分开期 然后将过去 慢慢温习 让我爱上你 那场悲剧 是你完美演出的一场戏 宁愿心碎哭泣 再狠狠忘记 你爱
epoch 600,perplexity 1.421720,time 0.49sec
- 分开 一直两双三仪四颗 连成线背著背默默许下心愿 看远方的星是否听的见 手牵手 一步两步三步四步望著天 
- 不分开吗 我叫你爸 你打我妈 这样对吗干嘛这样 何必让酒牵鼻子走 瞎 说不能 Ch我抬起头 有我去对医药箱
epoch 700,perplexity 1.359437,time 0.50sec
- 分开 距今已经三千七百多   纪录那最原始的美丽 纪录第一次

## 相邻采样进行训练

In [20]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                        vocab_size, ctx, corpus_indices, idx_to_char,
                        char_to_idx, False, num_epochs, num_steps, lr,
                        clipping_theta, batch_size, pred_period, pred_len,
                        prefixes)

KeyboardInterrupt: 

## 没有充分训练的模型好像一直在重复预测，好像是一直记住了前面的词
- 是否可以考虑，加入遗忘机制？
- 使用Relu作为激活函数之后好像收敛速度加快了很多